In [2]:
import pandas as pd 
df = pd.read_csv('imdb_indonesian_movies_2.csv')

In [6]:
df.head(5)

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [7]:
# membuat dummy variable dari genre menggunakan one hot encoding
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df,category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru.head()

,judul_film,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0


In [12]:
# memisahkan label dan fitur dan menjadikannya array numpy
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama','Horor','Komedi','Laga','Romantis']].values

In [13]:
#split data training dan testing
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(sinopsis, label, test_size=0.2)

In [14]:
# tokenisasi dan padding sinopsis
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenisasi sinopsis training dan testing
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)

#membuat array dari data yang sudah ditokenisasi
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

#menyamakan panjang array/sequence
train_padded = pad_sequences(train_sequences)
test_padded = pad_sequences(test_sequences)

In [18]:
#arsitektru model
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(62, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [20]:
#training model
history = model.fit(train_padded, y_train, validation_data=(test_padded, y_test), epochs=10)

Epoch 1/10
26/26 [==============================] - 18s 678ms/step - loss: 0.9688 - accuracy: 0.6045 - val_loss: 1.8336 - val_accuracy: 0.2786
Epoch 2/10
26/26 [==============================] - 17s 670ms/step - loss: 0.7627 - accuracy: 0.6679 - val_loss: 2.3428 - val_accuracy: 0.3483
Epoch 3/10
26/26 [==============================] - 19s 734ms/step - loss: 0.5454 - accuracy: 0.7376 - val_loss: 2.8785 - val_accuracy: 0.3184
Epoch 4/10
26/26 [==============================] - 18s 697ms/step - loss: 0.4840 - accuracy: 0.7811 - val_loss: 3.3374 - val_accuracy: 0.2985
Epoch 5/10
26/26 [==============================] - 18s 684ms/step - loss: 0.3867 - accuracy: 0.8582 - val_loss: 3.3128 - val_accuracy: 0.2935
Epoch 6/10
26/26 [==============================] - 18s 673ms/step - loss: 0.2521 - accuracy: 0.9204 - val_loss: 3.8682 - val_accuracy: 0.3134
Epoch 7/10
26/26 [==============================] - 18s 692ms/step - loss: 0.1345 - accuracy: 0.9639 - val_loss: 4.9873 - val_accuracy: 0.2786